In [1]:
import pandas as pd
import numpy as np

In [2]:
weight = [63, 21, 2, 32, 13, 80, 19, 37, 56, 41, 14, 8, 32, 42, 7]
value = [13, 2, 20, 10, 7, 14, 7, 2, 2, 4, 16, 17, 17, 3, 21]
objects = pd.DataFrame({'weights': weight, 'values': value})
limit_total_weight = 275

In [3]:
class Backpack:
  def __init__(self, array):
    self.inside = array
    self.total_weight = sum(objects.iloc[self.decode(array), 0])
    self.total_value = sum(objects.iloc[self.decode(array), 1])

  def decode(self, array):
    indexes = []
    for i in range(len(array)):
      if array[i] == 1:
        indexes.append(i)
    return indexes
  
  def mutate(self, gene):
    if self.inside[gene] == 0:
      copy_inside = self.inside.copy()
      copy_inside[gene] = 1
      sum_weight = sum(objects.iloc[self.decode(copy_inside), 0])
      if sum_weight <= 275:
        self.inside = copy_inside.copy()
        self.total_weight = sum(objects.iloc[self.decode(copy_inside), 0])
        self.total_value = sum(objects.iloc[self.decode(copy_inside), 1])
    else:
      self.inside[gene] = 0
      self.total_weight = sum(objects.iloc[self.decode(self.inside), 0])
      self.total_value = sum(objects.iloc[self.decode(self.inside), 1])

    return self
      

In [4]:
#The inputs: pulation_size for size of population
#reproduction_rate for number of pairs there will be for reproduction
#mutation_rate for percentage of mutations
class Population:
  def __init__(self, population_size, reproduction_rate, mutation_rate):
    self.population_size = population_size
    self.reproduction_rate = reproduction_rate
    self.mutation_rate = mutation_rate
    self.populate()

  def populate(self):
    self.population = []
    for i in range(self.population_size):
      actual_gene = np.random.choice([1, 0], size=(15), p=[2./3, 1./3])
      actual_gene = Backpack(actual_gene)
      if actual_gene.total_weight <= 275:
        self.population.append(actual_gene)
      else:
        while actual_gene.total_weight > 275:
          actual_gene = np.random.choice([1, 0], size=(15), p=[2./3, 1./3])
          actual_gene = Backpack(actual_gene)
        if actual_gene.total_weight <= 275:
          self.population.append(actual_gene)
  
  def select_parents(self):
    pairs = []
    for pair in range(self.reproduction_rate):
      all_fitness = [self.population[i].total_value for i in range(self.population_size)]
      fitness_chose = all_fitness.copy()
      sum_fitness = sum(all_fitness)
      accumulator = 0
      chosen_number = np.random.randint(1, sum_fitness, 1)
      for i in range(len(fitness_chose)):
        accumulator += fitness_chose[i]
        if accumulator >= chosen_number:
          first = fitness_chose[i]
          fitness_chose.pop(i)
          sum_fitness -= first
          break
      accumulator = 0
      chosen_number = np.random.randint(1, sum_fitness, 1)
      for i in range(len(fitness_chose)):
        accumulator += fitness_chose[i]
        if accumulator >= chosen_number:
          second = fitness_chose[i]
          fitness_chose.pop(i)
          sum_fitness -= first
          break
      pairs.append([all_fitness.index(first), all_fitness.index(second)])
    return pairs

  def reproduction(self, parents):
    all_sons = []
    for pair in parents:
      genomes = np.array(self.population)[pair]
      cut = np.random.randint(1, 14)
      genome1, genome2 = self.cross_over(genomes, cut)
      genome1 = Backpack(genome1)
      genome2 = Backpack(genome2)
      if genome1.total_weight <= 275 : all_sons.append(genome1)
      if genome2.total_weight <= 275 : all_sons.append(genome2) 
    
    return all_sons[:self.reproduction_rate]

  def cross_over(self, genomes, cut):
    parent1, parent2 = genomes
    son1 = parent1.inside[:cut].copy() 
    son1 = np.append(son1, parent2.inside[cut:].copy())

    son2 = parent1.inside[cut:].copy()
    son2 = np.append(son2, parent2.inside[:cut].copy())

    return (son1, son2)

  def mutation(self, genomes):
    all_genomes = []
    for genome in genomes:
      if np.random.randint(1, 100) < self.mutation_rate:
        genome = genome.mutate(np.random.randint(0, 15))
      all_genomes.append(genome)
    
    return all_genomes
  
  def bests_fitness(self, number_greater_values):
    all_fitness = [self.population[i].total_value for i in range(self.population_size)]
    index_best = all_fitness.index(max(all_fitness))
    best_solution = self.population[index_best]
    all_fitness.sort(reverse=True)
    
    return all_fitness[:number_greater_values], best_solution

In [5]:
#Genetic Algorithm
epochs = 1000
besties = 1
population = Population(100, 100, 5)

best_value, best_solution = population.bests_fitness(1)

actual_epoch = 0
for actual_epoch in range(epochs):
  actual_value, actual_solution = population.bests_fitness(1)
  if best_value < actual_value:
    best_solution = actual_solution
    best_value = actual_value

  parents = population.select_parents()
  genomes = population.reproduction(parents)
  genomes = population.mutation(genomes)

  population.population = genomes



In [8]:
print(f'Best value is {best_value}, and this itens {best_solution.decode(best_solution.inside)}, with this weight {best_solution.total_weight}')

Best value is [142], and this itens [0, 2, 3, 4, 5, 6, 10, 11, 12, 14], with this weight 270
